<a href="https://colab.research.google.com/github/KonstantinSV/text_recognition/blob/main/%D0%A0%D0%B0%D1%81%D0%BF%D0%BE%D0%B7%D0%BD%D0%B0%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%B0_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **Проект: определение рускоязычных токсичных комментариев**

Цель - обучить модель рпределяющую является комментарий токсичным или нет.

Данные для обучения:

Датасет на Kaggle - https://www.kaggle.com/blackmoon/russian-language-toxic-comments

# Предварительная обработка данных

In [ ]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/projects/project_text_recognition/labeled.csv')

In [ ]:
# Размеры DataFrame
data.shape

(14412, 2)

In [ ]:
# Первые пять строк DataFrame
data.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


Выделение данных для обучения и  целевого признака.

In [ ]:
# Данные
base = data.comment

# Целевой признак
target = data.toxic.values

In [ ]:
# Сравним соотношение меток класс 0 и класса 1 
print(data.toxic.value_counts())

0.0    9586
1.0    4826
Name: toxic, dtype: int64


Выделяем столбец, в котором будет указано количество восклицательных знаков в тексте.

In [ ]:
exclamations = base.apply(lambda x: x.count('!')).rename('num_exclamations')

Очистим текст от символов. 

In [ ]:
def clear_text(text):
    return re.sub(r'[\W]\s*', ' ', text).lower()

Проверка функции.

In [ ]:
# Возьмём произвольный текст
arbitrary_text = data.at[2020, "comment"]
print(f'Исходный текст:\n\n{arbitrary_text}\n')
print(f"Очищенный текст:\n\n{clear_text(arbitrary_text)}")

Исходный текст:

Вольнова же повесили на параше, ты про что?


Очищенный текст:

вольнова же повесили на параше ты про что 


In [ ]:
# Сохраним очищенный текст в переменной base
base = base.apply(clear_text)

# Обучение моделей

In [ ]:
import nltk
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from functools import partial
from scipy import sparse

In [ ]:
# Сохраним в переменную stop_en стоп-слова
nltk.download('stopwords')
nltk.download('wordnet')
stop_en = nltk.corpus.stopwords.words('russian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
# Функция для лемматизации текста
def lemmatization(text, lemm_func):
  return ' '.join(list(map(lemm_func, text.split())))

In [ ]:
# Функция обучает модель на данных, преобразованных с помощью объекта vectorizer
# Функция принимает данные лемматизирует их (если нужно), векторизует тексты в корпусе, выполняет кросс-валидацию модели, измеряет качество предсказаний на тестовой выборке.
def train_pipeline(model, vectorizer, x, y, with_exclamations=False, lemm_func=None, **kwargs):
    global exclamations
    if lemm_func:
        x = x.copy().apply(lemmatization, lemm_func=lemm_func)
        
    # Делим на выборки 3:1
    x_train, x_test, y_train, y_test, exc_train, exc_test = train_test_split(x, y, exclamations, random_state=0, test_size=.25)
    # Проверяем качество деления
    print('train_test_split: ')
    for x in (x_train, x_test, y_train, y_test, exc_train, exc_test):
        print(x.shape)
    # Создаём объект-векторизатор
    vectorizer = vectorizer(**kwargs)
    # Преобразуем корпус
    x_train = vectorizer.fit_transform(x_train)
    x_test = vectorizer.transform(x_test)
    # Если используем восклицательные знаки, добавим один столбец к матрице
    if with_exclamations:
        x_train = sparse.hstack([x_train, exc_train.values.reshape(-1, 1)], format='csr')
        x_test = sparse.hstack([x_test, exc_test.values.reshape(-1, 1)], format='csr')
    display(x_train)
    # Запишем результат кросс-валидации с метрикой F1
    print('cross-validating: ')
    output = cross_validate(model, x_train, y_train, cv=3, n_jobs=-1, scoring='f1')
    output['cv_score'] = np.mean(output['test_score'])
    output['fit_time'] = np.mean(output['fit_time'])
    del output['test_score']
    # Добавим F1 на тестовой выборке
    model.fit(x_train, y_train)
    output['test_score'] = f1_score(y_test, model.predict(x_test))
    del output['score_time']
    return output

In [ ]:
# Сохраним в переменную данные для обучения, целевой признак и стоп-слова
train_vectorized = partial(train_pipeline, x=base, y=target, stop_words=stop_en)

# Кодирование признаков способом мешок слов (Bag of words)

 **Логистическая регрессия.**

In [ ]:
# Обучение модели с базовыми параметрами
train_vectorized(LogisticRegression(class_weight='balanced'), vectorizer=CountVectorizer)

train_test_split: 
(10809,)
(3603,)
(10809,)
(3603,)
(10809,)
(3603,)


<10809x56801 sparse matrix of type '<class 'numpy.int64'>'
	with 173410 stored elements in Compressed Sparse Row format>

cross-validating: 


{'cv_score': 0.7587701230514657,
 'fit_time': 0.9655281702677408,
 'test_score': 0.7768940979489327}

In [ ]:
# Обучение модели со стеммингом
train_vectorized(LogisticRegression(class_weight='balanced'), lemm_func=SnowballStemmer('russian').stem, vectorizer=CountVectorizer)

train_test_split: 
(10809,)
(3603,)
(10809,)
(3603,)
(10809,)
(3603,)


<10809x28522 sparse matrix of type '<class 'numpy.int64'>'
	with 177779 stored elements in Compressed Sparse Row format>

cross-validating: 


{'cv_score': 0.7977692199878877,
 'fit_time': 0.5511995156606039,
 'test_score': 0.8091286307053941}

Пока что у нас нет веской причины использовать стемминг. Узнаем, насколько хорошо сработает лемматизация.

In [ ]:
# Обучение модели с лемматизацией
train_vectorized(LogisticRegression(class_weight='balanced'), lemm_func=WordNetLemmatizer().lemmatize, vectorizer=CountVectorizer)

train_test_split: 
(10809,)
(3603,)
(10809,)
(3603,)
(10809,)
(3603,)


<10809x56773 sparse matrix of type '<class 'numpy.int64'>'
	with 173387 stored elements in Compressed Sparse Row format>

cross-validating: 


{'cv_score': 0.758946829435359,
 'fit_time': 0.9382328987121582,
 'test_score': 0.7755443886097153}

**Наивный Байес**

In [ ]:
# Обучение Байеса
train_vectorized(BernoulliNB(), vectorizer=CountVectorizer)

train_test_split: 
(10809,)
(3603,)
(10809,)
(3603,)
(10809,)
(3603,)


<10809x56801 sparse matrix of type '<class 'numpy.int64'>'
	with 173410 stored elements in Compressed Sparse Row format>

cross-validating: 


{'cv_score': 0.3595160933246548,
 'fit_time': 0.015131711959838867,
 'test_score': 0.498159509202454}

Промежуточный вывод:
1. Байес работает быстрее логистической регрессии, но результаты логистической регрессии лучше чем Байеса.
2. Обучение логистической регрессии со стеммингом дало наилучший результат.


# ТF-IDF-кодирование признаков.

In [ ]:
# Обучение модели с базовыми параметрами
train_vectorized(LogisticRegression(class_weight='balanced'), vectorizer=TfidfVectorizer)

train_test_split: 
(10809,)
(3603,)
(10809,)
(3603,)
(10809,)
(3603,)


<10809x56801 sparse matrix of type '<class 'numpy.float64'>'
	with 173410 stored elements in Compressed Sparse Row format>

cross-validating: 


{'cv_score': 0.7706427965643624,
 'fit_time': 0.3012491861979167,
 'test_score': 0.7855626326963906}

In [ ]:
# Обучение модели с восклицательными знаками
train_vectorized(LogisticRegression(class_weight='balanced'), vectorizer=TfidfVectorizer, with_exclamations=True)

train_test_split: 
(10809,)
(3603,)
(10809,)
(3603,)
(10809,)
(3603,)


<10809x56802 sparse matrix of type '<class 'numpy.float64'>'
	with 174223 stored elements in Compressed Sparse Row format>

cross-validating: 


{'cv_score': 0.7700073689809251,
 'fit_time': 0.7238573233286539,
 'test_score': 0.7906382978723403}

Модель с восклицательными знаками показала себя лучше базовой


In [ ]:
# Обучение модели со стеммингом
train_vectorized(LogisticRegression(class_weight='balanced'), vectorizer=TfidfVectorizer, lemm_func=SnowballStemmer('russian').stem)

train_test_split: 
(10809,)
(3603,)
(10809,)
(3603,)
(10809,)
(3603,)


<10809x28522 sparse matrix of type '<class 'numpy.float64'>'
	with 177779 stored elements in Compressed Sparse Row format>

cross-validating: 


{'cv_score': 0.8156560792470743,
 'fit_time': 0.2217536767323812,
 'test_score': 0.8262833675564683}

Результаты слегка улучшились, но средняя F1 при кросс-валидации по-прежнему ниже требуемого значения.

In [ ]:
# Обучение модели с лемматизацией
train_vectorized(LogisticRegression(class_weight='balanced'), vectorizer=TfidfVectorizer, lemm_func=WordNetLemmatizer().lemmatize, with_exclamations=True)

train_test_split: 
(10809,)
(3603,)
(10809,)
(3603,)
(10809,)
(3603,)


<10809x56774 sparse matrix of type '<class 'numpy.float64'>'
	with 174200 stored elements in Compressed Sparse Row format>

cross-validating: 


{'cv_score': 0.7701167073881269,
 'fit_time': 0.6302946408589681,
 'test_score': 0.7911527009783071}

При кодировании ТF-IDF, модель со стеммингом показала результаты хуже чем с мешком слов.

Выводы и наблюдения:

Эффективнее остальных себя показал TfidfVectorizer, потому что с этим способом кодирования признаков F1 при кросс-валидации и F1 на тестовой выборке показывают самые лучшие результаты. 
Стемминг улучшил результаты в как с мешком слов, так и с TF-IDF.
Лемматизация показала себя хуже стемминга в обоих случаях.
Восклицательные знаки как признак не сыграли большой роли для качества предсказаний.

Лучший результат на данном этапе дают логистическая регрессия и TF-IDF со стеммингом.

# 3. Подбор гиперпараметров
Было применено два способа преобразования текстов для машинного обучения, и наиболее эффективным вариантом себя показала логистическая регрессия и TF-IDF со стеммингом. Для улучшения качества модели оптимизируем гиперпараметры.

In [ ]:
# Деление на обучающую и тестовую выборки 
xtr_final, xts_final, ytr_final, yts_final = train_test_split(base, target, random_state=0, test_size=0.2)
for df in (xtr_final, xts_final, ytr_final, yts_final):
    print(df.shape)

(11529,)
(2883,)
(11529,)
(2883,)


In [ ]:
# Проведение стемминга
def preprocessor(text, lemm_func):
  return ' '.join(list(map(lemm_func, text.split())))

xtr_final = xtr_final.copy().apply(preprocessor, lemm_func=SnowballStemmer('russian').stem)
xts_final = xts_final.copy().apply(preprocessor, lemm_func=SnowballStemmer('russian').stem)

In [ ]:
# Векторизация текстов
cvect = TfidfVectorizer(stop_words=stop_en)
xtr_final = cvect.fit_transform(xtr_final)
xts_final = cvect.transform(xts_final)

Лемматизацию и восклицательные знаки делать не будем. Эти операции не будут иметь большого смысла.

In [ ]:
# Создание GridSearch
grid = GridSearchCV(LogisticRegression(),
                        {'class_weight': ['balanced', None],
                         'solver': ['liblinear', 'newton-cg', 'sag', 'saga', 'lbfgs'],
                         'C': np.arange(0.1, 1.1, 0.1)}, scoring='f1', cv=3)

In [ ]:
# Обучение GridSearch
grid.fit(xtr_final, ytr_final)

GridSearchCV(cv=3, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]),
                         'class_weight': ['balanced', None],
                         'solver': ['liblinear', 'newton-cg', 'sag', 'saga',
                                    'lbfgs']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
  

In [ ]:
# Лучшие гиперпараметры подобранные GridSearch
grid.best_params_

{'C': 1.0, 'class_weight': 'balanced', 'solver': 'saga'}

In [ ]:
# Лучший результат кросс-валидации
grid.best_score_

0.8177946323065487

In [ ]:
# F1 на тестовой выборке
f1_score(yts_final, grid.predict(xts_final))

0.8292433537832311